In [1]:
!nvidia-smi

Tue May 31 11:13:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import shutil
from glob import glob
import os
from natsort import natsorted
import pandas as pd

In [5]:
!unzip -qq /content/drive/MyDrive/4-1_Capstone/CapstoneDataset/images_concatenate.zip -d /content/capstone_images
!unzip -qq /content/drive/MyDrive/4-1_Capstone/AI_HubDataset/images2000.zip -d /content/aihub_images

# !unzip -qq /content/drive/MyDrive/4-1_Capstone/LeafGAN/masked_images/low_performace_labels.zip -d /content/mask_images
!unzip -qq "/content/drive/MyDrive/4-1_Capstone/AI_HubDataset/clear_파 정상(예쁜)_mask.zip" -d /content/mask_images
!unzip -qq "/content/drive/MyDrive/4-1_Capstone/AI_HubDataset/clear_파 정상(예쁜).zip" -d /content/mask_images

replace /content/capstone_images/0_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### $len(masked\_images) < len(images)$ ==> segment가 별로인 이미지는 drop했기 때문

In [6]:
capstone_images = glob("/content/capstone_images/*")
aihub_images    = glob("/content/aihub_images/images2000/*")
images = capstone_images + aihub_images

mask_images = os.listdir("/content/mask_images")

images     = list(filter(lambda f : f.split('/')[-1] in mask_images, images))

mask_images = [os.path.join("/content/mask_images", f) for f in mask_images]

In [7]:
!git clone https://github.com/IyatomiLab/LeafGAN
%cd /content/LeafGAN
!pip install -r requirements.txt -qq
!pip install dominate -qq

Cloning into 'LeafGAN'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 239 (delta 51), reused 51 (delta 51), pack-reused 176
Receiving objects: 100% (239/239), 5.21 MiB | 20.35 MiB/s, done.
Resolving deltas: 100% (127/127), done.
/content/LeafGAN
     |████████████████████████████████| 676 kB 12.0 MB/s 
     |████████████████████████████████| 47.8 MB 1.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
ERROR: Could not find a version that satisfies the requirement skimage>=0.16.1 (from versions: 0.0)
ERROR: No matching distribution found for skimage>=0.16.1


In [8]:
    # discriminator lr *= 0.1
!cp /content/drive/MyDrive/4-1_Capstone/temp/leaf_gan_model.py /content/LeafGAN/models/leaf_gan_model.py
    # 필요없는 이미지 저장하지 않음(real, fake만 저장)
!cp /content/drive/MyDrive/4-1_Capstone/temp/visualizer.py /content/LeafGAN/util/visualizer.py

import sys
sys.path.append("/content/drive/MyDrive/4-1_Capstone")

from utils import get_disease_healthy_map 
disease_healthy_map = get_disease_healthy_map()

In [9]:
import shutil
def prepare_mask():
    os.mkdir("/content/dataset/trainA_mask")
    os.mkdir("/content/dataset/trainB_mask")
    os.mkdir("/content/dataset/testA_mask")
    os.mkdir("/content/dataset/testB_mask")
    
    SRC = "/content/mask_images"
    REF = ["/content/dataset/trainA", "/content/dataset/trainB", \
           "/content/dataset/testA", "/content/dataset/testB"]
    # DST = ["/content/dataset/trainA_mask", "/content/dataset/trainB_mask"]
    
    for ref in REF:
        images = os.listdir(ref)
        for image in images:
            src = os.path.join(SRC, image)
            DST = ref + '_mask'
            dst = os.path.join(DST, image)
            shutil.copy(src, dst)
    
def initGAN(disease_label):
    global images, SAMPLES
    healthy_label = disease_healthy_map[disease_label]

    healthy_images = list(filter(lambda f : f.split('/')[-1].split('_')[0] == healthy_label, images))
    disease_images = list(filter(lambda f : f.split('/')[-1].split('_')[0] == disease_label, images))
    
    if os.path.exists("/content/dataset"):
        os.system("rm -rf /content/dataset")

    os.mkdir("/content/dataset")
    os.mkdir("/content/dataset/trainA")
    os.mkdir("/content/dataset/testA")
    os.mkdir("/content/dataset/trainB")
    os.mkdir("/content/dataset/testB")


    healthy_images = np.array(healthy_images)
    disease_images = np.array(disease_images)

    from sklearn.model_selection import train_test_split
    healthy_idx = list(range(len(healthy_images)))
    disease_idx = list(range(len(disease_images)))
    
    train_A_i, test_A_i, _, _ = train_test_split(healthy_idx, healthy_idx, random_state=0)
    train_B_i, test_B_i, _, _ = train_test_split(disease_idx, disease_idx, random_state=0)
    
    import shutil
    for image in healthy_images[train_A_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/trainA", file_name)

        shutil.copy(image, dst)
        # shutil.move(image, dst)

    for image in healthy_images[test_A_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/testA", file_name)

        shutil.copy(image, dst)
        # shutil.move(image, dst)

    for image in disease_images[train_B_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/trainB", file_name)

        shutil.copy(image, dst)
        # shutil.move(image, dst)

    for image in disease_images[test_B_i]:
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/testB", file_name)

        shutil.copy(image, dst)
        # shutil.move(image, dst)

    # os.system("python prepare_mask.py --source /content/dataset --pretrain_path /content/drive/MyDrive/4-1_Capstone/LFLSeg_resnet101.pth --image_size 224")
    prepare_mask()


    shutil.copy("/content/drive/MyDrive/4-1_Capstone/temp/visualizer.py", \
                "/content/LeafGAN/util/visualizer.py")


    # os.system("python train.py --dataroot /content/dataset --name GAN" + disease_label + " --model leaf_gan --dataset_mode unaligned_masked")

def saveGAN(disease_label):
    # content의 학습된 모델을 drive로 옮기는 코드
    os.system("mkdir /content/drive/MyDrive/4-1_Capstone/GAN" + disease_label)
    os.system("cp /content/LeafGAN/checkpoints/GAN" + disease_label + "/latest_net_D_A.pth /content/drive/MyDrive/4-1_Capstone/GAN" + disease_label + "/latest_net_D_A.pth")
    os.system("cp /content/LeafGAN/checkpoints/GAN" + disease_label + "/latest_net_D_B.pth /content/drive/MyDrive/4-1_Capstone/GAN" + disease_label + "/latest_net_D_B.pth")
    os.system("cp /content/LeafGAN/checkpoints/GAN" + disease_label + "/latest_net_G_A.pth /content/drive/MyDrive/4-1_Capstone/GAN" + disease_label + "/latest_net_G_A.pth")
    os.system("cp /content/LeafGAN/checkpoints/GAN" + disease_label + "/latest_net_G_B.pth /content/drive/MyDrive/4-1_Capstone/GAN" + disease_label + "/latest_net_G_B.pth")

    output_filename = "/content/drive/MyDrive/4-1_Capstone/GAN" + disease_label + "/training_images"
    dir_name = "/content/LeafGAN/checkpoints/GAN" + disease_label + "/web/images"
    shutil.make_archive(output_filename, 'zip', dir_name)

def saveALL(disease_label):
    # 모든 모델, 이미지 drive로 옮기는 코드
    from glob import glob

    models = glob(f"/content/LeafGAN/checkpoints/GAN{disease_label}/*")
    models

    for model in models:
        model_name = model.split('/')[-1].split('_net_')[-1]
        # G_A가 아닌 모델은 삭제
        if model_name in ['G_B.pth', 'D_A.pth', 'D_B.pth']:
            os.remove(model)

    import shutil
    # drive로 바로 압축하는 것은 오래걸리기에 content에서 압축하고 파일을 옮김
    src = f"/content/LeafGAN/checkpoints/GAN{disease_label}"
    ref = f"/content/GAN{disease_label}"
    dst = f"/content/drive/MyDrive/4-1_Capstone/GAN{disease_label}/GAN{disease_label}"

    shutil.make_archive(ref, 'zip', src)
    if not os.path.exists(f"/content/drive/MyDrive/4-1_Capstone/GAN{disease_label}"):
        os.mkdir(f"/content/drive/MyDrive/4-1_Capstone/GAN{disease_label}")

    shutil.copy(ref + '.zip', dst + '.zip')

# GAN 뒤에 하드 코딩 해야됨

In [10]:
disease_label = '45'
initGAN(disease_label)
!python train.py --dataroot /content/dataset --name GAN45 --model leaf_gan --dataset_mode unaligned_masked --batch_size 16 --netG 'unet_256'
saveALL(disease_label)
# saveGAN(disease_label)

ValueError: ignored